<a href="https://colab.research.google.com/github/amritavarshini04/Genai_projects/blob/main/Customer_Feedback_Analyser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

data={
    "review_id" : [1,2,3],
    "review_text" : [
        "Got it delivered quick and the product is amazing!",
        "The service was not good, and I got a wrong item.",
        "Okay product, but the packaging was not very good."
    ]
}
df=pd.DataFrame(data)
df.head()

,review_id,review_text
0,1,Got it delivered quick and the product is amaz...
1,2,"The service was not good, and I got a wrong item."
2,3,"Okay product, but the packaging was not very g..."


Run sentimental analysis, the hugging face

In [6]:
from transformers import pipeline

sentiment_pipeline=pipeline("sentiment-analysis")
df["sentiment"]=df["review_text"].apply(lambda x: sentiment_pipeline(x)[0])
df.head()

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


,review_id,review_text,sentiment
0,1,Got it delivered quick and the product is amaz...,"{'label': 'POSITIVE', 'score': 0.9998822212219..."
1,2,"The service was not good, and I got a wrong item.","{'label': 'NEGATIVE', 'score': 0.9997468590736..."
2,3,"Okay product, but the packaging was not very g...","{'label': 'NEGATIVE', 'score': 0.9988971948623..."


Summarize long feedback

In [9]:
summarizer = pipeline("summarization")
df['summary']=df['review_text'].apply(lambda x: summarizer(x, max_length=30, min_length=5, do_sample=False)[0]['summary_text'])
df.head()

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cpu
Your max_length is set to 30, but your input_length is only 12. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)
Your max_length is set to 30, but your input_length is only 15. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Your max_length is set to 30, but your input_length is only 13. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)


,review_id,review_text,sentiment,summary
0,1,Got it delivered quick and the product is amaz...,"{'label': 'POSITIVE', 'score': 0.9998822212219...",Got it delivered quick and the product is ama...
1,2,"The service was not good, and I got a wrong item.","{'label': 'NEGATIVE', 'score': 0.9997468590736...","The service was not good, and I got a wrong i..."
2,3,"Okay product, but the packaging was not very g...","{'label': 'NEGATIVE', 'score': 0.9988971948623...","Okay product, but packaging was not very good ."


Generating AI replies using Gemini API

In [14]:
import google.generativeai as genai
from google.colab import userdata

try:
  API_KEY=userdata.get('GOOGLE_API_KEY')
  genai.configure(api_key=API_KEY)
  model=genai.GenerativeModel('gemini-pro')
except Exception as e:
  print(f"Error configuring Gemini API: {e}")
  print("Please make sure you have added your API key to Colab secrets with the name 'GOOGLE_API_KEY'.")

def generate_reply(review):
  if 'model' not in locals():
    return "Error: Gemini API not configured."
  prompt = f"You are a customer care agent. Respond politely to this customer feedback: {review}"
  try:
    response = model.generate_content(prompt)
    return response.text
  except Exception as e:
    return f"Error generating reply: {e}"

df['ai_reply'] = df['review_text'].apply(generate_reply)
df.head()

Error configuring Gemini API: Secret GOOGLE_API_KEY does not exist.
Please make sure you have added your API key to Colab secrets with the name 'GOOGLE_API_KEY'.


,review_id,review_text,sentiment,summary,ai_reply
0,1,Got it delivered quick and the product is amaz...,"{'label': 'POSITIVE', 'score': 0.9998822212219...",Got it delivered quick and the product is ama...,Error: Gemini API not configured.
1,2,"The service was not good, and I got a wrong item.","{'label': 'NEGATIVE', 'score': 0.9997468590736...","The service was not good, and I got a wrong i...",Error: Gemini API not configured.
2,3,"Okay product, but the packaging was not very g...","{'label': 'NEGATIVE', 'score': 0.9988971948623...","Okay product, but packaging was not very good .",Error: Gemini API not configured.


Visualize results, in streamlit dashboard

In [18]:
!pip install streamlit

In [17]:
import streamlit as st
import matplotlib.pyplot as plt

st.title("Customer Feedback Analysis")

# Count sentiment labels
sentiment_counts = df['sentiment'].apply(lambda x: x['label']).value_counts()

# Display sentiment counts as a bar chart
st.bar_chart(sentiment_counts)

st.subheader("Individual Reviews and AI Replies")
for index, row in df.iterrows():
  st.write(f"**Review ID:** {row['review_id']}")
  st.write(f"**Review:** {row['review_text']}")
  st.write(f"**Sentiment:** {row['sentiment']['label']} (Score: {row['sentiment']['score']:.2f})")
  st.write(f"**Summary:** {row['summary']}")
  st.write(f"**AI Reply:** {row['ai_reply']}")
  st.write("---")

2025-06-06 14:39:36.926 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-06 14:39:41.774 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-06-06 14:39:41.784 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-06 14:39:46.919 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-06 14:39:46.940 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-06 14:39:46.945 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-06 14:39:46.952 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-06 14:39:46.960 Thread 'MainThread': mi